In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from preprocess import load_frame

# df = pd.read_csv("DATASET.CSV", parse_dates=["Date"], index_col="Date")
df = pd.read_csv("dataset.csv")
df.index = pd.DatetimeIndex(df.datestamp)
df.index.name = "datestampindex"
# df = load_frame()

df.datestamp = pd.to_datetime(df.datestamp)

# df['Date'] = pd.to_datetime(df.index)
## df[['Close']].plot()
df[['close']].plot()
df.head()

import plotly.express as px
# fig = px.line(x=df.index, y=df.close, log_y=True)
# fig


In [ ]:
# debugging indexSelector, works well with DateScale only?
from bqplot import pyplot as plt, Figure, Axis
import bqplot as bq
from bqplot.interacts import IndexSelector, FastIntervalSelector
from bqplot.interacts import BrushIntervalSelector
from ipywidgets import ToggleButtons, VBox, HTML
import ipywidgets as widgets

class Slicer:
    
    db_index = HTML(value='[]')
    
    def selector_plot(self, df, x_col, y_col):

        def index_change_callback(change):
            if len(change.new) == 2:
                self.start_date = pd.to_datetime(change.new[0])
                self.end_date = pd.to_datetime(change.new[1])
                self.db_index.value = 'The selected date is ' + str(change.new)
                # self.sub_plot_fig = self.sub_plot()

        self.x_col = x_col
        self.y_col = y_col
        self.df = df
        
        # x_sc = bq.LinearScale()
        x_sc = bq.DateScale()
        # y_sc = bq.LogScale()
        y_sc = bq.LinearScale()

        x_ax = Axis(label=x_col, scale=x_sc)
        x_ay = Axis(label=y_col, scale=y_sc, orientation='vertical')
        lc = bq.Lines(x=df[x_col], y=df[y_col], scales={'x': x_sc, 'y': y_sc})
        
        selector = BrushIntervalSelector(scale=x_sc, marks=[lc])    
        selector.observe(index_change_callback, names=['selected'])
        self.selector_fig = Figure(marks=[lc], axes=[x_ax, x_ay], interaction=selector)
        return self.selector_fig

    def sub_plot(self):
        x_col = self.x_col
        y_col = self.y_col
        
        if hasattr(self, "start_date"):
            print (type(self.start_date))
            print (self.start_date)
            self.df_slice = self.df[self.start_date:self.end_date]
        else:
            self.df_slice = self.df
        
        # x_sc = bq.LinearScale()
        x_sc = bq.DateScale()
        # y_sc = bq.LogScale()
        y_sc = bq.LinearScale()

        x_ax = Axis(label=x_col, scale=x_sc)
        # x_ay = Axis(label=(y_col), scale=y_sc, orientation='vertical')
        x_ay = Axis(label=y_col, scale=y_sc, orientation='vertical')
        lc = bq.Lines(x=self.df_slice[x_col], y=self.df_slice[y_col], scales={'x': x_sc, 'y': y_sc})

        self.sub_plot_fig = Figure(marks=[lc], axes=[x_ax, x_ay])
        return self.sub_plot_fig

slicer = Slicer()
# slicer.selector_plot(df, "Date", "Close")
slicer.selector_plot(df, "datestamp", "close")

VBox([slicer.db_index, slicer.selector_fig])

In [ ]:
slicer.sub_plot()
slicer.df_slice.to_csv("dataset.csv", index=False)
